In [1]:
import numpy as np
from pyscf import gto, scf, fci
from tabulate import tabulate
from moha.hamiltonians import HamHub
from utils import Vee
from RDMFS import MU_U2RDM
import inspect 
import scipy 
#from numba.openmp import openmp_context as openmp

In [2]:
inspect.getfile(HamHub)

'/home/jerhard/1RDMFT/lib/python3.12/site-packages/moha/hamiltonians.py'

In [3]:
N = 2
norbs = N
nelec = N
system = [('H1', 'H2', 1)]
print(system)
connectivity = np.array([[0, 1],
                         [1, 0]])
U=2
t=-1
E0=-1
hubbard = HamHub(connectivity,alpha=t, beta=E0, u_onsite=U*np.ones((N)),
                      sym=8)

h_0 = hubbard.generate_zero_body_integral()
h_1 = hubbard.generate_one_body_integral(basis='spinorbital basis', dense=True)
h2 = hubbard.generate_two_body_integral(sym=8,basis='spinorbital basis',
                                                dense=True)

h2 = np.transpose(h2, (0, 2, 1, 3))
h2_hub_ab = h2[0:N,0:N,N:2*N,N:2*N]
h2_hub_aa = h2[0:N,0:N,0:N,0:N]
h2_hub_bb = h2[N:2*N,N:2*N,N:2*N,N:2*N]
h_1 = (h_1[0:N,0:N], h_1[N:2*N,N:2*N])
h2_hub = (h2_hub_aa, h2_hub_ab, h2_hub_bb)

cisolver = fci.direct_uhf.FCI()
cisolver.max_cycle = 100
cisolver.conv_tol = 1e-8
e, ci = cisolver.kernel(h_1, h2_hub, N, (N//2, N//2), ecore=h_0)
print(e)
dm1, dm2  = cisolver.make_rdm12s(ci, N, (N//2, N//2))

print(e-np.trace(h_1[0]@dm1[0])-np.trace(h_1[1]@dm1[1]), e)
# 2 E_0 + U
print(Vee(h2_hub,dm2), 2*E0+.5*U-np.sqrt(4*t**2+.25*U**2))

[('H1', 'H2', 1)]
-3.2360679774997854
0.5527864045000468 -3.2360679774997854
0.5527864045000423 -3.23606797749979
